In [17]:
import obspy
from obspy.io.sac.sactrace import SACTrace
from obspy.io.sac.util import get_sac_reftime
import os
import glob
import shutil
from obspy.clients.iris import Client
from obspy.taup import TauPyModel
import pandas as pd
import re

client = Client()
hk_model = 'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\hk.npz'
model = TauPyModel(model= hk_model)

In [2]:
ver_dir = r"C:\Users\youzh\Documents\My documents(not synced)\sP_Workflow\Download\Verified"
os.chdir(r"C:\Users\youzh\Documents\My documents(not synced)\sP_Workflow\Download\Verified")
df = pd.read_csv(f"{ver_dir}/verified.txt", names=["evt_id","sta", "Tp", 
                                                   "p_len","TsPmP", "sPmP_len",
                                                   "TsPn", "sPn_len", "quality", "evt_num"], header = None)

In [3]:
df

,evt_id,sta,Tp,p_len,TsPmP,sPmP_len,TsPn,sPn_len,quality,evt_num
0,39909423,CI.HOL.,15.558,0.3,18.173,0.3,nan,0.4,a,20220112002458770
1,39909511,AZ.LVA2.,3.723,0.2,11.662,0.3,nan,0.4,b,20220112021208390
2,39909511,AZ.SMER.,12.748,0.2,16.068,0.3,nan,0.4,b,20220112021208390
3,39909511,BC.RMX.,16.035,0.3,19.083,0.3,18.483,0.3,a,20220112021208390
4,39909663,CI.EML.,18.840,0.4,23.763,0.4,22.793,0.4,a,20220112090603170
...,...,...,...,...,...,...,...,...,...,...
96,40180784,CI.IKP.,13.600,0.4,15.700,0.4,nan,0.3,c,20220209121834630
97,40182952,CI.SRT.,4.950,0.4,11.700,0.4,nan,0.3,c,20220211121842170
98,40183048,CI.SRT.,4.920,0.4,12.000,0.4,nan,0.3,b,20220211141550450
99,40183056,CI.CCA.,15.000,0.4,18.200,0.4,nan,0.3,a,20220211142113660


In [5]:
os.getcwd()

'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\Download\\Verified'

In [4]:
import os
import re

folder_path = 'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\Download\\SAC'

# define the regex pattern to match the folder names
#pattern = re.compile(r'^2022[01][0-9]{11}$')

# get a list of all folders in the parent folder
folders = os.listdir(folder_path)

# filter the folders that match the regex pattern and fall within the specified range
matched_folders = []
for folder in folders:
    folder_int = int(folder)
    if 20220112002458770 < folder_int < 20220211232548160:
            matched_folders.append(os.path.join(folder_path, folder))

# print the list of matched folders
print(matched_folders[0])

C:\Users\youzh\Documents\My documents(not synced)\sP_Workflow\Download\SAC\20220112021208390


In [15]:
print(matched_folders[0].split("\\")[-1])

20220112021208390


In [18]:
os.listdir(matched_folders[0])

['arrivals.txt',
 'AZ.CRY..BHE.SAC',
 'AZ.CRY..BHE.SAC.bp',
 'AZ.CRY..BHE.SAC.bp.cut',
 'AZ.CRY..BHN.SAC',
 'AZ.CRY..BHN.SAC.bp',
 'AZ.CRY..BHN.SAC.bp.cut',
 'AZ.CRY..BHR.SAC.bp.cut',
 'AZ.CRY..BHT.SAC.bp.cut',
 'AZ.CRY..BHZ.SAC',
 'AZ.CRY..BHZ.SAC.bp',
 'AZ.CRY..BHZ.SAC.bp.cut',
 'AZ.LVA2..BHE.SAC',
 'AZ.LVA2..BHE.SAC.bp',
 'AZ.LVA2..BHE.SAC.bp.cut',
 'AZ.LVA2..BHN.SAC',
 'AZ.LVA2..BHN.SAC.bp',
 'AZ.LVA2..BHN.SAC.bp.cut',
 'AZ.LVA2..BHR.SAC.bp.cut',
 'AZ.LVA2..BHT.SAC.bp.cut',
 'AZ.LVA2..BHZ.SAC',
 'AZ.LVA2..BHZ.SAC.bp',
 'AZ.LVA2..BHZ.SAC.bp.cut',
 'AZ.RDM..BHE.SAC',
 'AZ.RDM..BHE.SAC.bp',
 'AZ.RDM..BHE.SAC.bp.cut',
 'AZ.RDM..BHN.SAC',
 'AZ.RDM..BHN.SAC.bp',
 'AZ.RDM..BHN.SAC.bp.cut',
 'AZ.RDM..BHR.SAC.bp.cut',
 'AZ.RDM..BHT.SAC.bp.cut',
 'AZ.RDM..BHZ.SAC',
 'AZ.RDM..BHZ.SAC.bp',
 'AZ.RDM..BHZ.SAC.bp.cut',
 'AZ.SMER..BHE.SAC',
 'AZ.SMER..BHE.SAC.bp',
 'AZ.SMER..BHE.SAC.bp.cut',
 'AZ.SMER..BHN.SAC',
 'AZ.SMER..BHN.SAC.bp',
 'AZ.SMER..BHN.SAC.bp.cut',
 'AZ.SMER..BHR.SAC.bp.cut',
 'AZ.S

In [5]:
bad_files = []

for folder in matched_folders:
    saved_evts = df["evt_num"].values
    evt_num = int(folder.split("\\")[-1])
    files = os.listdir(folder)
    if evt_num in saved_evts:
        saved_stations = df[df["evt_num"] == evt_num]["sta"].values
        stations = glob.glob(f"{folder}/*Z.SAC.bp.cut")
        for station in stations:
            station_name = " "+".".join(station.split("\\")[-1].split(".")[0:2])+"."
#             print(saved_stations[0])
#             print(station)
#             adsfasd
            if station_name not in saved_stations:
                bad_files.append(station)
print(len(bad_files))

5313


In [58]:
print(bad_files)

['C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\Download\\SAC\\20220112021208390\\AZ.CRY..BHZ.SAC.bp.cut', 'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\Download\\SAC\\20220112021208390\\AZ.RDM..BHZ.SAC.bp.cut', 'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\Download\\SAC\\20220112021208390\\AZ.SND..BHZ.SAC.bp.cut', 'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\Download\\SAC\\20220112021208390\\AZ.TRO..BHZ.SAC.bp.cut', 'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\Download\\SAC\\20220112021208390\\BC.TKX..HHZ.SAC.bp.cut', 'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\Download\\SAC\\20220112021208390\\CI.ACP..BHZ.SAC.bp.cut', 'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\Download\\SAC\\20220112021208390\\CI.AVM..BHZ.SAC.bp.cut', 'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\Download\\SAC\\20220112021208390\\

In [11]:
evt_dir  = 'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\Download\\events.txt'

events = pd.read_csv(evt_dir, header = None, delim_whitespace=True)


# Function to remove non-numerical symbols from a string
def remove_non_numerical(s):
    return re.sub('[^0-9]', '', s)

# Apply the function to the 'col1' column of the DataFrame
events[3] = events[3].apply(remove_non_numerical)
events

,0,1,2,3,4,5,6,7,8,9
0,39900807,eq,l,20220102001614080,32.2623,-115.3403,4.03,2.28,l,1.0
1,39901391,eq,l,20220102232745920,33.6480,-116.7418,15.41,2.36,l,1.0
2,39901519,eq,l,20220103023448200,35.2840,-119.3907,17.73,3.92,l,1.0
3,39902431,eq,l,20220104090504520,32.9600,-115.8628,7.95,2.00,l,1.0
4,39902479,eq,l,20220104124219810,34.1240,-117.5977,11.83,2.08,l,1.0
...,...,...,...,...,...,...,...,...,...,...
509,40292048,eq,l,20220627041323370,33.8412,-118.5292,10.77,2.27,l,1.0
510,40292088,eq,l,20220627065127410,32.6447,-115.7207,9.34,2.07,l,1.0
511,40292496,eq,l,20220627195131740,35.6527,-117.4580,1.33,2.10,l,1.0
512,40293376,eq,l,20220629041936690,33.1813,-115.6073,2.96,3.02,l,1.0


In [14]:
os.path.basename((os.path.split(file_path)[0]))
#os.path.split(file_path)[0]

'20220211142113660'

In [15]:
dst_folder = 'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\BAD_SAC_files'
#os.listdir(dst_folder)
for file_path in bad_files:
    file_name = os.path.basename(file_path)
    evt_num = os.path.basename((os.path.split(file_path)[0]))
    dst_path = os.path.join(dst_folder, evt_num)
    if not os.path.exists(dst_path):
        os.makedirs(dst_path)
    dst_file_path = os.path.join(dst_path, file_name)
    
    shutil.copy(file_path, dst_file_path)

In [19]:
data_dir = 'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\BAD_SAC_files'
data_dest = 'C:\\Users\\youzh\\Documents\\My documents(not synced)\\sP_Workflow\\BAD_SAC_for_readin'
for ev in os.listdir(data_dir):
    inpath = os.path.join(data_dir, ev)
    print(ev)
    
    for sac in glob.glob(f"{inpath}/*.bp.cut"):
        
        insac = sac#os.path.join(inpath, sac)
        
        
        st = obspy.read(insac)
        #print(st)
        fname = ev+'.' + st[0].stats.network + '.' + st[0].stats.station + st[0].stats.location + '.BHZ'
#             if os.path.exists(f"{data_dest}/{fname}"):
#                 continue
        evt_index = events.loc[events[3] == ev].index[0]
        #print(evt_index)
        st[0].stats.sac['nevid'] = str(events[0][evt_index])

        # write sP arrival
        sachd = st[0].stats.sac
        distaz = client.distaz(sachd["stla"], sachd["stlo"], sachd["evla"], sachd["evlo"])
        gcarc = distaz['distance']
        arrivals = model.get_travel_times(source_depth_in_km=sachd["evdp"],
                              distance_in_degree=gcarc, phase_list=['sPvmP'])

        TsPmP = arrivals[0].time#df_verified["TsPmP"][evt_index]
        #print(TsPmP)
        st[0].stats.sac["t2"] = TsPmP + sachd["o"]
        st[0].stats.sac["kt2"] = "sPmP"
        # resample
        #st.resample(sample_rate, no_filter="False")
        #splt = sac.split('.')
        destin = os.path.join(data_dest, ev)
        if not os.path.exists(destin):
            os.makedirs(destin)
        st[0].write(f"{destin}/{fname}.SAC.bp", format='SAC')

.ipynb_checkpoints
20220112021208390


C:\Users\youzh\anaconda3\envs\pygmt\Lib\site-packages\obspy\io\sac\arrayio.py:507: UserWarning: Non-integers may be truncated: nevid = 39909511
  warnings.warn(msg.format(hdr, value))


20220112090603170


C:\Users\youzh\anaconda3\envs\pygmt\Lib\site-packages\obspy\io\sac\arrayio.py:507: UserWarning: Non-integers may be truncated: nevid = 39909663
  warnings.warn(msg.format(hdr, value))


20220113032108850


...

20220205234904960


C:\Users\youzh\anaconda3\envs\pygmt\Lib\site-packages\obspy\io\sac\arrayio.py:507: UserWarning: Non-integers may be truncated: nevid = 40177968
  warnings.warn(msg.format(hdr, value))


20220206232447510


C:\Users\youzh\anaconda3\envs\pygmt\Lib\site-packages\obspy\io\sac\arrayio.py:507: UserWarning: Non-integers may be truncated: nevid = 40178728
  warnings.warn(msg.format(hdr, value))


RemoteDisconnected: Remote end closed connection without response